# AI Curriculum Planner - Curriculum Analysis

This notebook provides comprehensive analysis of the AI academic advising system's curriculum graph structure, course prerequisites, domain statistics, and interactive exploration tools.

## Overview
- **Curriculum Graph**: Visual representation of courses and prerequisites
- **Course Analysis**: Detailed examination of individual courses
- **Domain Statistics**: Distribution and relationships across academic domains
- **Prerequisites Analysis**: Dependency analysis and critical path identification
- **Interactive Exploration**: Tools for exploring the course network

## Course Data References and Sources

### Curriculum Design References

The computer science curriculum used in this simulation is based on:

1. **ACM Computing Curricula 2020** - Association for Computing Machinery
   - Reference: *Computing Curricula 2020: Paradigms for Global Computing Education*
   - Used for domain classification and course progression standards

2. **IEEE Computer Society Curriculum Guidelines** - Institute of Electrical and Electronics Engineers
   - Reference: *Computer Science Curricula 2013: Curriculum Guidelines for Undergraduate Degree Programs*
   - Applied for prerequisite structure and credit hour standards

3. **Real University Computer Science Programs** analyzed:
   - Stanford University CS Curriculum (2023)
   - MIT EECS Course Sequences
   - UC Berkeley CS Program Structure
   - Carnegie Mellon Computer Science Department

### 5.2 Course Prerequisites and Structure

The prerequisite relationships were designed following standard academic progressions:

- **Foundation Courses**: CS101 (Intro Programming), MATH101 (Calculus), CS102 (Data Structures)
- **Intermediate Prerequisites**: Most 200-level courses require 100-level foundations
- **Advanced Prerequisites**: 300+ level courses require specific intermediate courses
- **Cross-domain Prerequisites**: AI/ML courses require statistics; Systems courses require low-level programming

### 5.3 Realistic Course Naming and Content

Course codes and names follow standard computer science nomenclature:
- CS: Computer Science core courses
- MATH: Mathematical foundations  
- DS: Data Science specialization
- SEC: Security/Cybersecurity
- SE: Software Engineering

In [43]:
# Import Required Libraries
import sys
import os
sys.path.append('../')  # Add parent directory to path

# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Project modules
from src.curriculum_graph import CurriculumGraph, create_sample_curriculum
from src.student_simulation import StudentSimulator, create_student_cohort
from src.constraints import AcademicConstraints, create_constraint_validator
from src.visualization import CurriculumVisualizer, create_visualizer

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ All libraries imported successfully")
print("✓ Project modules loaded")
print("✓ Plotting configuration set")

✓ All libraries imported successfully
✓ Project modules loaded
✓ Plotting configuration set


In [44]:
# Initialize Curriculum System
print("Initializing AI Curriculum Planner System...")

# Create curriculum graph
curriculum = create_sample_curriculum()
print(f"✓ Curriculum created with {len(curriculum.graph.nodes())} courses")

# Get basic statistics
stats = curriculum.get_stats()
print(f"✓ {stats['total_prerequisites']} prerequisite relationships")
print(f"✓ {len(stats['domains'])} academic domains")

# Create other components
constraints = create_constraint_validator(curriculum)
visualizer = create_visualizer(curriculum)

print("\n" + "="*50)
print("CURRICULUM OVERVIEW")
print("="*50)
print(f"Total Courses: {stats['total_courses']}")
print(f"Total Prerequisites: {stats['total_prerequisites']}")
print(f"Average Prerequisites per Course: {stats['avg_prerequisites']:.2f}")
print(f"Graph Complexity Score: {stats['complexity']}")
print("\nDomain Distribution:")
for domain, count in stats['domains'].items():
    print(f"  {domain}: {count} courses")

print("\n✓ System initialization complete!")

Initializing AI Curriculum Planner System...
✓ Curriculum created with 34 courses
✓ 37 prerequisite relationships
✓ 6 academic domains

CURRICULUM OVERVIEW
Total Courses: 34
Total Prerequisites: 37
Average Prerequisites per Course: 1.09
Graph Complexity Score: 71

Domain Distribution:
  AI: 6 courses
  Security: 4 courses
  Data Science: 6 courses
  Software Engineering: 9 courses
  Systems: 4 courses
  Theory: 5 courses

✓ System initialization complete!


## 1. Curriculum Graph Visualization

Let's explore the structure of our computer science curriculum as a directed graph where:
- **Nodes** represent courses with attributes (domain, difficulty, credits)
- **Edges** represent prerequisite relationships
- **Colors** indicate academic domains
- **Sizes** reflect course difficulty levels

In [45]:
# Create Clean Top-Down Curriculum Graph
print("Creating clean top-down curriculum visualization...")

# Get graph data
G = curriculum.graph

# Calculate course levels for hierarchical layout
course_levels = {}
topo_order = list(nx.topological_sort(G))
for node in topo_order:
    prereqs = curriculum.get_prerequisites(node)
    if not prereqs:
        course_levels[node] = 0  # Starting level
    else:
        max_prereq_level = max(course_levels.get(prereq, 0) for prereq in prereqs)
        course_levels[node] = max_prereq_level + 1

# Group courses by level
max_level = max(course_levels.values())
level_nodes = {level: [] for level in range(max_level + 1)}
for node, level in course_levels.items():
    level_nodes[level].append(node)

# Create clean top-down layout
pos = {}
y_spacing = -2.0  # Vertical spacing between levels (negative for top-down)
x_spacing = 1.5   # Horizontal spacing between nodes

for level, nodes in level_nodes.items():
    y = level * y_spacing  # Top-down arrangement
    num_nodes = len(nodes)
    
    # Center the nodes horizontally
    if num_nodes == 1:
        x_positions = [0]
    else:
        total_width = (num_nodes - 1) * x_spacing
        x_positions = [i * x_spacing - total_width / 2 for i in range(num_nodes)]
    
    # Sort nodes by domain for better visual grouping
    nodes_sorted = sorted(nodes, key=lambda n: curriculum.get_course_info(n).get('domain', 'ZZZ'))
    
    for i, node in enumerate(nodes_sorted):
        pos[node] = (x_positions[i], y)

# Prepare node styling
node_x, node_y, node_text, node_color, node_size = [], [], [], [], []

domain_colors = {
    'AI': '#E74C3C', 'Security': '#3498DB', 'Data Science': '#9B59B6',
    'Software Engineering': '#27AE60', 'Systems': '#F39C12', 'Theory': '#E67E22'
}

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    course_info = curriculum.get_course_info(node)
    domain = course_info.get('domain', 'Other')
    difficulty = course_info.get('difficulty', 'Intermediate')
    
    # Clean color scheme
    node_color.append(domain_colors.get(domain, '#95A5A6'))
    
    # Larger sizing to accommodate text
    size_map = {'Beginner': 55, 'Intermediate': 60, 'Advanced': 65}
    node_size.append(size_map.get(difficulty, 60))
    
    # Clean hover information
    prereqs = curriculum.get_prerequisites(node)
    level = course_levels.get(node, 0)
    
    text = (f"<b>{node}</b><br>"
            f"{course_info.get('name', 'Unknown')}<br>"
            f"Domain: {domain}<br>"
            f"Level: {level}<br>"
            f"Prerequisites: {', '.join(prereqs) if prereqs else 'None'}")
    node_text.append(text)

# Create simple edge traces
edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Create the plot
fig = go.Figure()

# Add clean edges
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=1.5, color='#BDC3C7'),
    hoverinfo='none',
    showlegend=False
))

# Add nodes
fig.add_trace(go.Scatter(
    x=node_x, y=node_y, 
    mode='markers+text',
    marker=dict(
        size=node_size, 
        color=node_color, 
        line=dict(width=2, color='white'),
        opacity=0.9
    ),
    text=[node for node in G.nodes()], 
    textposition="middle center",
    textfont=dict(size=11, color='white', family='Arial Black'), 
    hovertext=node_text,
    hoverinfo='text',
    showlegend=False
))

# Add level labels
for level in range(max_level + 1):
    if level_nodes[level]:  # Only add label if level has courses
        fig.add_annotation(
            x=min(pos[node][0] for node in level_nodes[level]) - 2,
            y=level * y_spacing,
            text=f"<b>Level {level}</b>",
            showarrow=False,
            font=dict(size=14, color='#2C3E50'),
            xanchor='right'
        )

# Add domain legend
legend_y = max_level * y_spacing - 1
legend_x = max(node_x) + 1
for i, (domain, color) in enumerate(domain_colors.items()):
    fig.add_trace(go.Scatter(
        x=[legend_x], y=[legend_y - i * 0.3],
        mode='markers+text',
        marker=dict(size=15, color=color, line=dict(width=1, color='white')),
        text=domain,
        textposition="middle right",
        textfont=dict(size=10, color='#2C3E50'),
        hoverinfo='skip',
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title=dict(
        text="Computer Science Curriculum - Top-Down Hierarchical View",
        font=dict(size=20, color='#2C3E50'),
        x=0.5
    ),
    showlegend=False,
    hovermode='closest',
    xaxis=dict(
        showgrid=False, 
        zeroline=False, 
        showticklabels=False,
        range=[min(node_x) - 3, max(node_x) + 4]
    ),
    yaxis=dict(
        showgrid=False, 
        zeroline=False, 
        showticklabels=False,
        range=[min(node_y) - 1, max(node_y) + 1]
    ),
    width=1200, 
    height=800,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=50, r=200, t=80, b=50)
)

fig.show()

print("✓ Clean top-down curriculum graph created!")
print("✓ Courses are organized by academic level (top to bottom)")
print("✓ Colors represent different domains")
print("✓ Hover over nodes for detailed information")

Creating clean top-down curriculum visualization...


✓ Clean top-down curriculum graph created!
✓ Courses are organized by academic level (top to bottom)
✓ Colors represent different domains
✓ Hover over nodes for detailed information


## 2. Course Analysis and Statistics

Let's dive deeper into the individual courses and their characteristics.

In [46]:
# Course Analysis and Statistics
print("Analyzing course characteristics...")

# Create course dataframe
course_data = []
for node in curriculum.graph.nodes():
    course_info = curriculum.get_course_info(node)
    prereqs = curriculum.get_prerequisites(node)
    all_prereqs = curriculum.get_all_prerequisites(node)
    
    course_data.append({
        'Course': node,
        'Name': course_info.get('name', 'Unknown'),
        'Domain': course_info.get('domain', 'Other'),
        'Difficulty': course_info.get('difficulty', 'Intermediate'),
        'Credits': course_info.get('credits', 3),
        'Direct_Prerequisites': len(prereqs),
        'Total_Prerequisites': len(all_prereqs),
        'Prerequisites_List': ', '.join(prereqs) if prereqs else 'None'
    })

courses_df = pd.DataFrame(course_data)

# Display basic statistics
print(f"📊 COURSE STATISTICS")
print(f"Total Courses: {len(courses_df)}")
print(f"Average Credits: {courses_df['Credits'].mean():.1f}")
print(f"Average Direct Prerequisites: {courses_df['Direct_Prerequisites'].mean():.1f}")
print(f"Average Total Prerequisites: {courses_df['Total_Prerequisites'].mean():.1f}")

# Show distribution by domain
print(f"\n📈 DOMAIN DISTRIBUTION")
domain_counts = courses_df['Domain'].value_counts()
for domain, count in domain_counts.items():
    percentage = (count / len(courses_df)) * 100
    print(f"  {domain}: {count} courses ({percentage:.1f}%)")

# Show distribution by difficulty
print(f"\n🎯 DIFFICULTY DISTRIBUTION")
difficulty_counts = courses_df['Difficulty'].value_counts()
for difficulty, count in difficulty_counts.items():
    percentage = (count / len(courses_df)) * 100
    print(f"  {difficulty}: {count} courses ({percentage:.1f}%)")

# Display first few courses with details
print(f"\n📋 SAMPLE COURSES")
display_cols = ['Course', 'Name', 'Domain', 'Difficulty', 'Direct_Prerequisites', 'Prerequisites_List']
print(courses_df[display_cols].head(10).to_string(index=False))

print("\n✓ Course analysis complete!")

Analyzing course characteristics...
📊 COURSE STATISTICS
Total Courses: 34
Average Credits: 3.0
Average Direct Prerequisites: 1.1
Average Total Prerequisites: 3.1

📈 DOMAIN DISTRIBUTION
  Software Engineering: 9 courses (26.5%)
  Data Science: 6 courses (17.6%)
  AI: 6 courses (17.6%)
  Theory: 5 courses (14.7%)
  Systems: 4 courses (11.8%)
  Security: 4 courses (11.8%)

🎯 DIFFICULTY DISTRIBUTION
  Advanced: 19 courses (55.9%)
  Intermediate: 13 courses (38.2%)
  Beginner: 2 courses (5.9%)

📋 SAMPLE COURSES
 Course                  Name               Domain   Difficulty  Direct_Prerequisites Prerequisites_List
  CS101  Intro to Programming Software Engineering     Beginner                     0               None
  CS102       Data Structures Software Engineering Intermediate                     1              CS101
  CS201            Algorithms               Theory Intermediate                     2     CS102, MATH101
  CS210 Computer Architecture              Systems Intermediate     

In [47]:
# Create visualizations for course statistics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Domain Distribution', 'Difficulty Distribution', 
                   'Prerequisites Analysis', 'Credits Distribution'],
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "box"}, {"type": "histogram"}]]
)

# Domain distribution pie chart
domain_counts = courses_df['Domain'].value_counts()
fig.add_trace(go.Pie(
    labels=domain_counts.index, values=domain_counts.values,
    name="Domain Distribution", hole=0.3
), row=1, col=1)

# Difficulty distribution bar chart
difficulty_counts = courses_df['Difficulty'].value_counts()
fig.add_trace(go.Bar(
    x=difficulty_counts.index, y=difficulty_counts.values,
    name="Difficulty Distribution", marker_color='lightblue'
), row=1, col=2)

# Prerequisites analysis box plot
fig.add_trace(go.Box(
    y=courses_df['Direct_Prerequisites'], name="Direct",
    marker_color='lightgreen'
), row=2, col=1)
fig.add_trace(go.Box(
    y=courses_df['Total_Prerequisites'], name="Total",
    marker_color='orange'
), row=2, col=1)

# Credits distribution histogram
fig.add_trace(go.Histogram(
    x=courses_df['Credits'], name="Credits",
    marker_color='lightcoral', nbinsx=5
), row=2, col=2)

fig.update_layout(
    title_text="Course Statistics Dashboard",
    height=800, showlegend=False
)

fig.show()

# Additional analysis: Most connected courses
print("\n🔗 MOST CONNECTED COURSES (by prerequisites required)")
top_prereq_courses = courses_df.nlargest(10, 'Total_Prerequisites')[['Course', 'Name', 'Domain', 'Total_Prerequisites']]
print(top_prereq_courses.to_string(index=False))

print("\n🚪 ENTRY-LEVEL COURSES (no prerequisites)")
entry_courses = courses_df[courses_df['Direct_Prerequisites'] == 0][['Course', 'Name', 'Domain']]
print(entry_courses.to_string(index=False))


🔗 MOST CONNECTED COURSES (by prerequisites required)
Course                        Name       Domain  Total_Prerequisites
 CS312               Deep Learning           AI                    8
 CS313 Natural Language Processing           AI                    8
 CS314             Computer Vision           AI                    8
 CS311            Machine Learning           AI                    7
 CS315                    Robotics           AI                    7
SEC302                Cryptography     Security                    5
 CS310     Artificial Intelligence           AI                    4
 DS301                 Data Mining Data Science                    4
 DS302          Big Data Analytics Data Science                    4
SEC303            Network Security     Security                    4

🚪 ENTRY-LEVEL COURSES (no prerequisites)
 Course                 Name               Domain
  CS101 Intro to Programming Software Engineering
MATH101 Discrete Mathematics               Th

## 3. Prerequisites Dependency Analysis

Understanding the prerequisite dependencies is crucial for course scheduling and graduation planning.

In [48]:
# Prerequisites Dependency Analysis
print("Analyzing prerequisite dependencies...")

G = curriculum.graph

# Calculate various graph metrics
print("🔍 GRAPH ANALYSIS METRICS")
print(f"Graph Density: {nx.density(G):.3f}")
print(f"Is DAG (Directed Acyclic Graph): {nx.is_directed_acyclic_graph(G)}")
print(f"Number of Weakly Connected Components: {nx.number_weakly_connected_components(G)}")

# Find topological ordering (valid course sequence)
if nx.is_directed_acyclic_graph(G):
    topo_order = list(nx.topological_sort(G))
    print(f"✓ Topological ordering exists (no circular dependencies)")
    print(f"First 10 courses in valid sequence: {topo_order[:10]}")
else:
    print("⚠️ Circular dependencies detected!")

# Calculate course levels (minimum term to take each course)
course_levels = {}
for node in topo_order:
    prereqs = curriculum.get_prerequisites(node)
    if not prereqs:
        course_levels[node] = 1  # First term
    else:
        max_prereq_level = max(course_levels.get(prereq, 0) for prereq in prereqs)
        course_levels[node] = max_prereq_level + 1

# Add levels to dataframe
courses_df['Min_Term'] = courses_df['Course'].map(course_levels)

print(f"\n📊 COURSE LEVEL DISTRIBUTION")
level_counts = courses_df['Min_Term'].value_counts().sort_index()
for level, count in level_counts.items():
    print(f"  Term {level}: {count} courses")

# Find critical paths (longest prerequisite chains)
print(f"\n🛤️ CRITICAL PATHS ANALYSIS")
longest_paths = []
for node in G.nodes():
    if G.out_degree(node) == 0:  # Leaf nodes (advanced courses)
        try:
            # Find longest path to this node
            path_lengths = nx.single_source_shortest_path_length(G.reverse(), node)
            max_length = max(path_lengths.values())
            longest_paths.append((node, max_length))
        except:
            continue

longest_paths.sort(key=lambda x: x[1], reverse=True)
print("Courses with longest prerequisite chains:")
for course, length in longest_paths[:10]:
    course_info = curriculum.get_course_info(course)
    print(f"  {course} ({course_info.get('name', 'Unknown')}): {length} prerequisites")

# Identify bottleneck courses (many courses depend on them)
bottlenecks = []
for node in G.nodes():
    dependent_courses = len(list(nx.descendants(G, node)))
    if dependent_courses > 0:
        bottlenecks.append((node, dependent_courses))

bottlenecks.sort(key=lambda x: x[1], reverse=True)
print(f"\n🔧 BOTTLENECK COURSES (enable many other courses)")
for course, dependents in bottlenecks[:10]:
    course_info = curriculum.get_course_info(course)
    print(f"  {course} ({course_info.get('name', 'Unknown')}): enables {dependents} courses")

Analyzing prerequisite dependencies...
🔍 GRAPH ANALYSIS METRICS
Graph Density: 0.033
Is DAG (Directed Acyclic Graph): True
Number of Weakly Connected Components: 1
✓ Topological ordering exists (no circular dependencies)
First 10 courses in valid sequence: ['CS101', 'MATH101', 'MATH201', 'MATH202', 'CS102', 'CS210', 'CS301', 'DS304', 'CS201', 'CS250']

📊 COURSE LEVEL DISTRIBUTION
  Term 1: 4 courses
  Term 2: 4 courses
  Term 3: 8 courses
  Term 4: 10 courses
  Term 5: 5 courses
  Term 6: 3 courses

🛤️ CRITICAL PATHS ANALYSIS
Courses with longest prerequisite chains:
  CS312 (Deep Learning): 5 prerequisites
  CS313 (Natural Language Processing): 5 prerequisites
  CS314 (Computer Vision): 5 prerequisites
  SEC302 (Cryptography): 4 prerequisites
  SEC303 (Network Security): 4 prerequisites
  SEC304 (Ethical Hacking): 4 prerequisites
  CS315 (Robotics): 3 prerequisites
  DS301 (Data Mining): 3 prerequisites
  DS302 (Big Data Analytics): 3 prerequisites
  SE301 (Advanced Programming): 3 pr

## 4. Interactive Course Explorer

Let's create interactive tools to explore specific courses and their relationships.

In [49]:
# Interactive Course Explorer Functions

def explore_course(course_code):
    """Detailed exploration of a specific course"""
    if course_code not in curriculum.graph.nodes():
        print(f"❌ Course {course_code} not found!")
        return
    
    course_info = curriculum.get_course_info(course_code)
    prereqs = curriculum.get_prerequisites(course_code)
    all_prereqs = curriculum.get_all_prerequisites(course_code)
    
    # Find courses that have this as prerequisite
    enables = [node for node in curriculum.graph.nodes() 
               if course_code in curriculum.get_prerequisites(node)]
    
    # Find all courses enabled by taking this course
    all_enables = list(nx.descendants(curriculum.graph, course_code))
    
    print(f"🎓 COURSE ANALYSIS: {course_code}")
    print(f"Name: {course_info.get('name', 'Unknown')}")
    print(f"Domain: {course_info.get('domain', 'Unknown')}")
    print(f"Difficulty: {course_info.get('difficulty', 'Unknown')}")
    print(f"Credits: {course_info.get('credits', 3)}")
    print(f"Minimum Term: {course_levels.get(course_code, 'Unknown')}")
    
    print(f"\n📋 PREREQUISITES")
    print(f"Direct Prerequisites ({len(prereqs)}): {', '.join(prereqs) if prereqs else 'None'}")
    print(f"All Prerequisites ({len(all_prereqs)}): {', '.join(sorted(all_prereqs)) if all_prereqs else 'None'}")
    
    print(f"\n🔓 ENABLES")
    print(f"Direct Enables ({len(enables)}): {', '.join(enables) if enables else 'None'}")
    print(f"All Enables ({len(all_enables)}): {', '.join(sorted(all_enables)[:10]) if all_enables else 'None'}")
    if len(all_enables) > 10:
        print(f"  ... and {len(all_enables) - 10} more courses")

def explore_domain(domain):
    """Explore all courses in a specific domain"""
    domain_courses = curriculum.get_courses_by_domain(domain)
    
    if not domain_courses:
        print(f"❌ No courses found in domain: {domain}")
        return
    
    print(f"🎯 DOMAIN ANALYSIS: {domain}")
    print(f"Total Courses: {len(domain_courses)}")
    
    # Group by difficulty
    domain_df = courses_df[courses_df['Domain'] == domain]
    difficulty_groups = domain_df.groupby('Difficulty')
    
    print(f"\n📊 BREAKDOWN BY DIFFICULTY")
    for difficulty, group in difficulty_groups:
        print(f"{difficulty}: {len(group)} courses")
        courses_list = group['Course'].tolist()
        print(f"  Courses: {', '.join(courses_list)}")
    
    # Show progression path
    domain_by_level = domain_df.groupby('Min_Term')['Course'].apply(list).to_dict()
    print(f"\n🛤️ SUGGESTED PROGRESSION PATH")
    for term in sorted(domain_by_level.keys()):
        courses = domain_by_level[term]
        print(f"Term {term}: {', '.join(courses)}")

def find_graduation_path(completed_courses=None):
    """Find a path to graduation"""
    if completed_courses is None:
        completed_courses = set()
    
    remaining = curriculum.get_graduation_path(completed_courses)
    
    print(f"🎓 GRADUATION PATH ANALYSIS")
    print(f"Completed Courses: {len(completed_courses)}")
    print(f"Remaining Courses: {len(remaining)}")
    
    if remaining:
        # Group by estimated term
        path_by_term = {}
        temp_completed = completed_courses.copy()
        
        for i, course in enumerate(remaining):
            term = (i // 4) + 1  # Assume 4 courses per term
            if term not in path_by_term:
                path_by_term[term] = []
            path_by_term[term].append(course)
        
        print(f"\n📅 SUGGESTED GRADUATION SCHEDULE")
        for term in sorted(path_by_term.keys()):
            courses = path_by_term[term]
            print(f"Term {term}: {', '.join(courses)}")
            if term <= 3:  # Show details for first few terms
                for course in courses:
                    info = curriculum.get_course_info(course)
                    print(f"  - {course}: {info.get('name', 'Unknown')} ({info.get('domain', 'Unknown')})")

# Example usage
print("🔍 INTERACTIVE EXPLORATION TOOLS LOADED")
print("\nAvailable functions:")
print("  explore_course('CS311') - Detailed course analysis")
print("  explore_domain('AI') - Analyze all AI courses")
print("  find_graduation_path() - Generate graduation plan")
print("\nLet's try some examples:")

# Example explorations
print("\n" + "="*60)
explore_course('CS311')  # Machine Learning

print("\n" + "="*60)
explore_domain('AI')

print("\n" + "="*60)
# Example with some completed courses
sample_completed = {'CS101', 'CS102', 'MATH101', 'MATH201'}
find_graduation_path(sample_completed)

🔍 INTERACTIVE EXPLORATION TOOLS LOADED

Available functions:
  explore_course('CS311') - Detailed course analysis
  explore_domain('AI') - Analyze all AI courses
  find_graduation_path() - Generate graduation plan

Let's try some examples:

🎓 COURSE ANALYSIS: CS311
Name: Machine Learning
Domain: AI
Difficulty: Advanced
Credits: 3
Minimum Term: 5

📋 PREREQUISITES
Direct Prerequisites (3): MATH201, MATH202, CS310
All Prerequisites (7): CS101, CS102, CS201, CS310, MATH101, MATH201, MATH202

🔓 ENABLES
Direct Enables (3): CS312, CS313, CS314
All Enables (3): CS312, CS313, CS314

🎯 DOMAIN ANALYSIS: AI
Total Courses: 6

📊 BREAKDOWN BY DIFFICULTY
Advanced: 6 courses
  Courses: CS310, CS311, CS312, CS313, CS314, CS315

🛤️ SUGGESTED PROGRESSION PATH
Term 4: CS310
Term 5: CS311, CS315
Term 6: CS312, CS313, CS314

🎓 GRADUATION PATH ANALYSIS
Completed Courses: 4
Remaining Courses: 30

📅 SUGGESTED GRADUATION SCHEDULE
Term 1: CS230, DS302, MATH202, DS301
  - CS230: Database Systems (Data Science)
  -

## 5. Summary and Next Steps

### Key Findings

1. **Curriculum Structure**: Our computer science curriculum contains 32 courses across 6 domains with a well-structured prerequisite network.

2. **Domain Balance**: The curriculum provides good coverage across all major CS areas, with emphasis on practical skills (Software Engineering) and foundational theory.

3. **Prerequisite Dependencies**: The curriculum forms a proper DAG (Directed Acyclic Graph) with no circular dependencies, enabling clear progression paths.

4. **Critical Courses**: Foundational courses like CS101, CS102, and MATH101 are bottlenecks that enable many advanced courses.

### Recommendations for Academic Advisors

1. **Early Focus**: Students should prioritize foundational courses (CS101, CS102, MATH101) early as they unlock many advanced options.

2. **Domain Specialization**: Students can begin specializing in their area of interest (AI, Security, Data Science) from Term 3 onwards.

3. **Balanced Progression**: The curriculum allows for balanced progression with 3-4 courses per term leading to graduation in 8 terms.

### Next Steps

- **Student Simulation**: Analyze how different student profiles progress through this curriculum
- **Recommendation System**: Use this analysis to build personalized course recommendation algorithms
- **Optimization**: Identify opportunities to reduce bottlenecks and improve curriculum flow

---

*Continue to the next notebook: `student_simulation.ipynb` for student cohort analysis*